# This program is used for counting single words only.

In [1]:
import pymongo

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['newsTemp']   # Enter your database name
mycol = mydb['articles']     # Enter your collection name

In [3]:
records = mycol.find()

In [6]:
uniqueWordCount = {}
wordCount = {}

In [1]:
i = 0
for row in records:
    content = row['content']
    content = content.split()
    content = list(filter(lambda x: x.isalnum(), content))
    content = list(map(lambda x: x.lower(), content))
    if (content is None) or (len(content) == 0):
        content = ['']
        
    for word in content:
        if word == '':
            continue
        temp1 = wordCount.pop(word, False)
        if temp1 is False:
            wordCount[word] = {row['date']: 1}
        else:
            temp2 = temp1.pop(row['date'], False)
            if temp2 is False:
                temp1[row['date']] = 1
            else:
                temp1[row['date']] = temp2 + 1
            wordCount[word] = temp1
            
        temp = uniqueWordCount.pop(word, False)
        if temp is False:
            uniqueWordCount[word] = 1
        else:
            uniqueWordCount[word] = temp + 1
          
        
    # Just to check progress
    i += 1
    if i % 1000 == 0:
        print "Completed " + str(i)
    

NameError: name 'records' is not defined

In [9]:
print len(wordCount.keys())

233267


In [21]:
res = [None] * len(wordCount.keys())

In [22]:
i = 0
for word in wordCount.keys():
    tempObj = {
        "word": word,
        "countByDate": wordCount[word]
    }
    res[i] = tempObj
    i += 1

In [23]:
newDb = myclient['ngramCounts']
newCol = newDb['allNgrams']
x = newCol.insert_many(res)

In [24]:
print len(uniqueWordCount.keys())

233267


In [25]:
arr = [None] * len(uniqueWordCount.keys())
i = 0
for word in uniqueWordCount.keys():
    tempJson = {
        "word": word,
        "count": uniqueWordCount[word]
    }
    arr[i] = tempJson
    i += 1

In [26]:
newDb = myclient['uniqueWordCount']
newCol = newDb['uniqueCount']
x = newCol.insert_many(arr)

In [27]:
newDb = myclient['uniqueWordCount']
newCol = newDb['oneGram']
x = newCol.insert_many(arr)

In [28]:
newDb = myclient['ngramCount']
newCol = newDb['oneGram']
x = newCol.insert_many(res)